### Import Module

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from keras import optimizers
from keras.models import Sequential, load_model
from keras.layers import Dense,Activation, LSTM, Dropout, TimeDistributed, Flatten
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

from sklearn.utils import class_weight
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/jason/anaconda/lib/python3.6/site-packages/tensorflo

### Load Data

In [2]:
training_load = pd.read_csv('training_1961_2016.csv', delimiter=',')
testing_2017_load = pd.read_csv('testing_2017.csv', delimiter=',')
testing_2018_load = pd.read_csv('testing_2018.csv', delimiter=',')
testing_2019_load = pd.read_csv('testing_2019.csv', delimiter=',')

In [3]:
training_load.head(5)

,FirstYear_index,FirstYear_playerID,FirstYear_yearID,FirstYear_teamID,FirstYear_lgID,FirstYear_stint,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,...,SixthYear_HBP,SixthYear_SH,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age
0,41261,aaronha01,1961,ML1,NL,1.0,155.0,603.0,115.0,197.0,...,1.0,0.0,8.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,32.0
1,41959,aaronha01,1962,ML1,NL,1.0,156.0,592.0,127.0,191.0,...,0.0,0.0,6.0,11.0,Hank,Aaron,Henry Louis,180.0,72.0,33.0
2,42719,aaronha01,1963,ML1,NL,1.0,161.0,631.0,121.0,201.0,...,1.0,0.0,5.0,21.0,Hank,Aaron,Henry Louis,180.0,72.0,34.0
3,43471,aaronha01,1964,ML1,NL,1.0,145.0,570.0,103.0,187.0,...,2.0,0.0,3.0,14.0,Hank,Aaron,Henry Louis,180.0,72.0,35.0
4,44225,aaronha01,1965,ML1,NL,1.0,150.0,570.0,109.0,181.0,...,2.0,0.0,6.0,13.0,Hank,Aaron,Henry Louis,180.0,72.0,36.0


In [4]:
var_name = ['index', 'playerID', 'teamID', 'lgID', 'stint','nameFirst', 'nameLast', 'nameGiven']

First = []
for i in range(len(var_name)):
    First.append(f"FirstYear_{var_name[i]}")

Second = []
for i in range(len(var_name)):
    Second.append(f"SecondYear_{var_name[i]}")

Third = []
for i in range(len(var_name)):
    Third.append(f"ThirdYear_{var_name[i]}")
    
Fourth = []
for i in range(len(var_name)):
    Fourth.append(f"FourthYear_{var_name[i]}")
    
Fifth = []
for i in range(len(var_name)):
    Fifth.append(f"FifthYear_{var_name[i]}")
    
FileColumnName = First + Second + Third + Fourth + Fifth

In [5]:
x_train = training_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(training_load), 5, 21))
y_train = training_load['SixthYear_HR'].values
x_test_2017 = testing_2017_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2017_load), 5, 21))
y_test_2017 = testing_2017_load['SixthYear_HR'].values
x_test_2018 = testing_2018_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2018_load), 5, 21))
y_test_2018 = testing_2018_load['SixthYear_HR'].values
x_test_2019 = testing_2019_load.iloc[:,0:-29].drop(FileColumnName, axis=1).values.reshape((len(testing_2019_load), 5, 21))
y_test_2019 = testing_2019_load['SixthYear_HR'].values

In [6]:
x_train_2017 = np.concatenate((x_train, x_test_2017))
y_train_2017 = np.concatenate((y_train, y_test_2017))

In [245]:
testing_2018 = testing_2018_load.drop(FileColumnName, axis=1).copy()

### Models

In [7]:
model_A = Sequential()
model_A.add(LSTM(128, input_shape=(5, 21), return_sequences=True))
model_A.add(LSTM(128, return_sequences=True))
model_A.add(Dropout(rate=0.5))
model_A.add(Flatten())
model_A.add(Dense(1024, activation="relu"))
model_A.add(Dropout(rate=0.5))
model_A.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_A.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_A.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 5, 128)            76800     
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 128)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               328192    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                32832     
__________

In [8]:
model_B = Sequential()
model_B.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_B.add(LSTM(16, return_sequences=True))
model_B.add(LSTM(8, return_sequences=True))
model_B.add(Dropout(rate=0.5))
model_B.add(Flatten())
model_B.add(Dense(512, activation="relu"))
model_B.add(Dropout(rate=0.5))
model_B.add(Dense(64, activation="relu"))
model_B.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_B.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_B.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 5, 128)            76800     
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 128)            131584    
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 128)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 640)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              656384    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1025      
Total para

In [9]:
model_C = Sequential()
model_C.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_C.add(LSTM(32, return_sequences=True))
model_C.add(BatchNormalization())
model_C.add(Flatten())
model_C.add(Dense(512, activation="relu"))
model_C.add(BatchNormalization())
model_C.add(Dense(64, activation="relu"))
model_C.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_C.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_C.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 5, 32)             6912      
_________________________________________________________________
lstm_6 (LSTM)                (None, 5, 32)             8320      
_________________________________________________________________
lstm_7 (LSTM)                (None, 5, 32)             8320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 32)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               82432     
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
__________

In [10]:
model_D = Sequential()
model_D.add(LSTM(64, input_shape=(5, 21), return_sequences=True))
model_D.add(LSTM(64, return_sequences=True))
model_D.add(Dropout(rate=0.5))
model_D.add(TimeDistributed(Dense(10)))
model_D.add(Flatten())
model_D.add(Dense(512, activation="relu"))
model_D.add(Dropout(rate=0.5))
model_D.add(Dense(64, activation="relu"))
model_D.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_D.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_D.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5, 32)             6912      
_________________________________________________________________
lstm_9 (LSTM)                (None, 5, 16)             3136      
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 8)              800       
_________________________________________________________________
dropout_7 (Dropout)          (None, 5, 8)              0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 40)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               20992     
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
__________

In [11]:
model_E = Sequential()
model_E.add(LSTM(32, input_shape=(5, 21), return_sequences=True))
model_E.add(LSTM(32, return_sequences=True))
model_E.add(Dropout(rate=0.5))
model_E.add(Flatten())
model_E.add(Dense(512, activation="relu"))
model_E.add(Dropout(rate=0.5))
model_E.add(Dense(64, activation="relu"))
model_E.add(Dense(1, activation='relu'))
adam = optimizers.Adam(lr = 0.001)
model_E.compile(loss = "mse", optimizer=adam, metrics=["accuracy"])
model_E.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 5, 32)             6912      
_________________________________________________________________
lstm_12 (LSTM)               (None, 5, 32)             8320      
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 32)             128       
_________________________________________________________________
flatten_5 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 512)               82432     
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_13 (Dense)             (None, 64)                32832     
__________

### Training

In [20]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "A/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_A_stat = model_A.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 9828 samples, validate on 184 samples
Epoch 1/1000
9828/9828 [==============================] - 7s 743us/step - loss: 94.3408 - acc: 0.0403 - val_loss: 75.6924 - val_acc: 0.0380

Epoch 00001: val_acc improved from -inf to 0.03804, saving model to B/saved-model-001-0.0380.hdf5
Epoch 2/1000
9828/9828 [==============================] - 6s 629us/step - loss: 65.5720 - acc: 0.0575 - val_loss: 74.2934 - val_acc: 0.0217

Epoch 00002: val_acc did not improve
Epoch 3/1000
9828/9828 [==============================] - 6s 642us/step - loss: 59.8002 - acc: 0.0609 - val_loss: 57.0495 - val_acc: 0.0598

Epoch 00003: val_acc improved from 0.03804 to 0.05978, saving model to B/saved-model-003-0.0598.hdf5
Epoch 4/1000
9828/9828 [==============================] - 6s 643us/step - loss: 57.1254 - acc: 0.0692 - val_loss: 57.0367 - val_acc: 0.0815

Epoch 00004: val_acc improved from 0.05978 to 0.08152, saving model to B/saved-model-004-0.0815.hdf5
Epoch 5/1000
9828/9828 [============================

9828/9828 [==============================] - 6s 638us/step - loss: 50.5698 - acc: 0.0752 - val_loss: 50.5893 - val_acc: 0.0652

Epoch 00044: val_acc did not improve
Epoch 45/1000
9828/9828 [==============================] - 9s 915us/step - loss: 48.6926 - acc: 0.0830 - val_loss: 49.9918 - val_acc: 0.0598

Epoch 00045: val_acc did not improve
Epoch 46/1000
9828/9828 [==============================] - 7s 714us/step - loss: 48.1055 - acc: 0.0796 - val_loss: 49.9006 - val_acc: 0.0652

Epoch 00046: val_acc did not improve
Epoch 47/1000
9828/9828 [==============================] - 7s 717us/step - loss: 47.8429 - acc: 0.0868 - val_loss: 49.5960 - val_acc: 0.0543

Epoch 00047: val_acc did not improve
Epoch 48/1000
9828/9828 [==============================] - 7s 725us/step - loss: 48.8811 - acc: 0.0817 - val_loss: 52.0745 - val_acc: 0.0707

Epoch 00048: val_acc did not improve
Epoch 49/1000
9828/9828 [==============================] - 7s 669us/step - loss: 49.1202 - acc: 0.0763 - val_loss: 50.2


Epoch 00134: val_acc did not improve
Epoch 135/1000
9828/9828 [==============================] - 6s 628us/step - loss: 47.9713 - acc: 0.0758 - val_loss: 56.4540 - val_acc: 0.0543

Epoch 00135: val_acc did not improve
Epoch 136/1000
9828/9828 [==============================] - 6s 625us/step - loss: 47.4899 - acc: 0.0753 - val_loss: 50.1483 - val_acc: 0.0489

Epoch 00136: val_acc did not improve
Epoch 137/1000
9828/9828 [==============================] - 6s 630us/step - loss: 47.1124 - acc: 0.0790 - val_loss: 48.0484 - val_acc: 0.0870

Epoch 00137: val_acc did not improve
Epoch 138/1000
9828/9828 [==============================] - 6s 621us/step - loss: 48.4895 - acc: 0.0742 - val_loss: 49.2193 - val_acc: 0.1033

Epoch 00138: val_acc did not improve
Epoch 139/1000
9828/9828 [==============================] - 6s 628us/step - loss: 47.1428 - acc: 0.0762 - val_loss: 51.5422 - val_acc: 0.0652

Epoch 00139: val_acc did not improve
Epoch 140/1000
9828/9828 [==============================] - 6s


Epoch 00225: val_acc did not improve
Epoch 226/1000
9828/9828 [==============================] - 7s 713us/step - loss: 45.9432 - acc: 0.0749 - val_loss: 50.6962 - val_acc: 0.0435

Epoch 00226: val_acc did not improve
Epoch 227/1000
9828/9828 [==============================] - 6s 643us/step - loss: 46.2620 - acc: 0.0739 - val_loss: 53.5261 - val_acc: 0.0543

Epoch 00227: val_acc did not improve
Epoch 228/1000
9828/9828 [==============================] - 6s 625us/step - loss: 46.3825 - acc: 0.0763 - val_loss: 49.4705 - val_acc: 0.0489

Epoch 00228: val_acc did not improve
Epoch 229/1000
9828/9828 [==============================] - 6s 619us/step - loss: 46.3662 - acc: 0.0784 - val_loss: 49.4289 - val_acc: 0.0598

Epoch 00229: val_acc did not improve
Epoch 230/1000
9828/9828 [==============================] - 6s 617us/step - loss: 46.4471 - acc: 0.0776 - val_loss: 50.6876 - val_acc: 0.0598

Epoch 00230: val_acc did not improve
Epoch 231/1000
9828/9828 [==============================] - 6s


Epoch 00316: val_acc did not improve
Epoch 317/1000
9828/9828 [==============================] - 6s 623us/step - loss: 46.2702 - acc: 0.0753 - val_loss: 50.6686 - val_acc: 0.0707

Epoch 00317: val_acc did not improve
Epoch 318/1000
9828/9828 [==============================] - 6s 619us/step - loss: 46.5433 - acc: 0.0804 - val_loss: 50.7122 - val_acc: 0.0489

Epoch 00318: val_acc did not improve
Epoch 319/1000
9828/9828 [==============================] - 6s 622us/step - loss: 46.8934 - acc: 0.0741 - val_loss: 49.9151 - val_acc: 0.0489

Epoch 00319: val_acc did not improve
Epoch 320/1000
9828/9828 [==============================] - 7s 700us/step - loss: 46.1183 - acc: 0.0742 - val_loss: 49.6293 - val_acc: 0.0489

Epoch 00320: val_acc did not improve
Epoch 321/1000
9828/9828 [==============================] - 6s 654us/step - loss: 46.7018 - acc: 0.0767 - val_loss: 54.7872 - val_acc: 0.0543

Epoch 00321: val_acc did not improve
Epoch 322/1000
9828/9828 [==============================] - 6s

...


Epoch 00589: val_acc did not improve
Epoch 590/1000
9828/9828 [==============================] - 7s 687us/step - loss: 46.1062 - acc: 0.0781 - val_loss: 51.9295 - val_acc: 0.0815

Epoch 00590: val_acc did not improve
Epoch 591/1000
9828/9828 [==============================] - 6s 657us/step - loss: 46.4878 - acc: 0.0744 - val_loss: 52.4919 - val_acc: 0.0652

Epoch 00591: val_acc did not improve
Epoch 592/1000
9828/9828 [==============================] - 6s 640us/step - loss: 46.2884 - acc: 0.0710 - val_loss: 52.0272 - val_acc: 0.0489

Epoch 00592: val_acc did not improve
Epoch 593/1000
9828/9828 [==============================] - 6s 628us/step - loss: 46.0433 - acc: 0.0766 - val_loss: 52.3122 - val_acc: 0.0815

Epoch 00593: val_acc did not improve
Epoch 594/1000
9828/9828 [==============================] - 6s 627us/step - loss: 46.0421 - acc: 0.0740 - val_loss: 59.5204 - val_acc: 0.0326

Epoch 00594: val_acc did not improve
Epoch 595/1000
9828/9828 [==============================] - 6s


Epoch 00680: val_acc did not improve
Epoch 681/1000
9828/9828 [==============================] - 6s 622us/step - loss: 45.3650 - acc: 0.0756 - val_loss: 52.3267 - val_acc: 0.0707

Epoch 00681: val_acc did not improve
Epoch 682/1000
9828/9828 [==============================] - 6s 624us/step - loss: 45.6971 - acc: 0.0773 - val_loss: 53.8546 - val_acc: 0.0652

Epoch 00682: val_acc did not improve
Epoch 683/1000
9828/9828 [==============================] - 6s 621us/step - loss: 45.5423 - acc: 0.0768 - val_loss: 54.7482 - val_acc: 0.0543

Epoch 00683: val_acc did not improve
Epoch 684/1000
9828/9828 [==============================] - 7s 691us/step - loss: 46.4736 - acc: 0.0755 - val_loss: 51.3796 - val_acc: 0.0870

Epoch 00684: val_acc did not improve
Epoch 685/1000
9828/9828 [==============================] - 6s 657us/step - loss: 45.9529 - acc: 0.0751 - val_loss: 52.7821 - val_acc: 0.0489

Epoch 00685: val_acc did not improve
Epoch 686/1000
9828/9828 [==============================] - 6s


Epoch 00771: val_acc did not improve
Epoch 772/1000
9828/9828 [==============================] - 6s 624us/step - loss: 45.1010 - acc: 0.0769 - val_loss: 51.4850 - val_acc: 0.0761

Epoch 00772: val_acc did not improve
Epoch 773/1000
9828/9828 [==============================] - 6s 626us/step - loss: 45.7607 - acc: 0.0778 - val_loss: 54.5616 - val_acc: 0.0543

Epoch 00773: val_acc did not improve
Epoch 774/1000
9828/9828 [==============================] - 6s 624us/step - loss: 46.0378 - acc: 0.0753 - val_loss: 53.4950 - val_acc: 0.0707

Epoch 00774: val_acc did not improve
Epoch 775/1000
9828/9828 [==============================] - 6s 620us/step - loss: 46.4442 - acc: 0.0761 - val_loss: 51.0831 - val_acc: 0.0652

Epoch 00775: val_acc did not improve
Epoch 776/1000
9828/9828 [==============================] - 6s 623us/step - loss: 45.3702 - acc: 0.0749 - val_loss: 54.1589 - val_acc: 0.0543

Epoch 00776: val_acc did not improve
Epoch 777/1000
9828/9828 [==============================] - 6s


Epoch 00862: val_acc did not improve
Epoch 863/1000
9828/9828 [==============================] - 6s 645us/step - loss: 45.6984 - acc: 0.0806 - val_loss: 51.5582 - val_acc: 0.0435

Epoch 00863: val_acc did not improve
Epoch 864/1000
9828/9828 [==============================] - 6s 632us/step - loss: 45.6709 - acc: 0.0783 - val_loss: 51.5678 - val_acc: 0.0598

Epoch 00864: val_acc did not improve
Epoch 865/1000
9828/9828 [==============================] - 6s 621us/step - loss: 45.6010 - acc: 0.0747 - val_loss: 53.7031 - val_acc: 0.0707

Epoch 00865: val_acc did not improve
Epoch 866/1000
9828/9828 [==============================] - 6s 625us/step - loss: 45.8450 - acc: 0.0768 - val_loss: 51.6041 - val_acc: 0.0761

Epoch 00866: val_acc did not improve
Epoch 867/1000
9828/9828 [==============================] - 6s 626us/step - loss: 45.3001 - acc: 0.0758 - val_loss: 53.4661 - val_acc: 0.0870

Epoch 00867: val_acc did not improve
Epoch 868/1000
9828/9828 [==============================] - 6s


Epoch 00953: val_acc did not improve
Epoch 954/1000
9828/9828 [==============================] - 6s 626us/step - loss: 44.8997 - acc: 0.0757 - val_loss: 51.5626 - val_acc: 0.0598

Epoch 00954: val_acc did not improve
Epoch 955/1000
9828/9828 [==============================] - 6s 628us/step - loss: 45.0731 - acc: 0.0784 - val_loss: 51.1250 - val_acc: 0.0707

Epoch 00955: val_acc did not improve
Epoch 956/1000
9828/9828 [==============================] - 7s 702us/step - loss: 45.2540 - acc: 0.0786 - val_loss: 51.1855 - val_acc: 0.0598

Epoch 00956: val_acc did not improve
Epoch 957/1000
9828/9828 [==============================] - 6s 659us/step - loss: 44.7230 - acc: 0.0770 - val_loss: 55.6143 - val_acc: 0.0489

Epoch 00957: val_acc did not improve
Epoch 958/1000
9828/9828 [==============================] - 6s 617us/step - loss: 46.1079 - acc: 0.0781 - val_loss: 50.9739 - val_acc: 0.0652

Epoch 00958: val_acc did not improve
Epoch 959/1000
9828/9828 [==============================] - 6s

In [22]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "B/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_B_stat = model_B.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 9828 samples, validate on 184 samples
Epoch 1/1000
9828/9828 [==============================] - 4s 362us/step - loss: 128.7559 - acc: 0.0423 - val_loss: 97.7372 - val_acc: 0.0380

Epoch 00001: val_acc improved from -inf to 0.03804, saving model to D/saved-model-001-0.0380.hdf5
Epoch 2/1000
9828/9828 [==============================] - 2s 158us/step - loss: 91.4137 - acc: 0.0404 - val_loss: 103.8585 - val_acc: 0.0543

Epoch 00002: val_acc improved from 0.03804 to 0.05435, saving model to D/saved-model-002-0.0543.hdf5
Epoch 3/1000
9828/9828 [==============================] - 1s 152us/step - loss: 81.1630 - acc: 0.0456 - val_loss: 89.9108 - val_acc: 0.0380

Epoch 00003: val_acc did not improve
Epoch 4/1000
9828/9828 [==============================] - 2s 159us/step - loss: 75.4112 - acc: 0.0440 - val_loss: 89.9926 - val_acc: 0.0380

Epoch 00004: val_acc did not improve
Epoch 5/1000
9828/9828 [==============================] - 1s 151us/step - loss: 71.7311 - acc: 0.0496 - val_loss: 


Epoch 00044: val_acc did not improve
Epoch 45/1000
9828/9828 [==============================] - 1s 152us/step - loss: 55.1979 - acc: 0.0666 - val_loss: 61.7693 - val_acc: 0.0924

Epoch 00045: val_acc improved from 0.09239 to 0.09239, saving model to D/saved-model-045-0.0924.hdf5
Epoch 46/1000
9828/9828 [==============================] - 3s 259us/step - loss: 55.1185 - acc: 0.0664 - val_loss: 67.3912 - val_acc: 0.0707

Epoch 00046: val_acc did not improve
Epoch 47/1000
9828/9828 [==============================] - 2s 242us/step - loss: 54.8712 - acc: 0.0603 - val_loss: 65.0870 - val_acc: 0.0598

Epoch 00047: val_acc did not improve
Epoch 48/1000
9828/9828 [==============================] - 2s 156us/step - loss: 54.8558 - acc: 0.0615 - val_loss: 62.6417 - val_acc: 0.0489

Epoch 00048: val_acc did not improve
Epoch 49/1000
9828/9828 [==============================] - 2s 154us/step - loss: 53.6413 - acc: 0.0644 - val_loss: 60.7234 - val_acc: 0.0380

Epoch 00049: val_acc did not improve
Epo

9828/9828 [==============================] - 2s 186us/step - loss: 50.1420 - acc: 0.0719 - val_loss: 54.8406 - val_acc: 0.0870

Epoch 00134: val_acc did not improve
Epoch 135/1000
9828/9828 [==============================] - 2s 205us/step - loss: 50.1215 - acc: 0.0694 - val_loss: 54.6395 - val_acc: 0.0924

Epoch 00135: val_acc did not improve
Epoch 136/1000
9828/9828 [==============================] - 2s 169us/step - loss: 49.8256 - acc: 0.0677 - val_loss: 61.2985 - val_acc: 0.0543

Epoch 00136: val_acc did not improve
Epoch 137/1000
9828/9828 [==============================] - 2s 167us/step - loss: 50.0446 - acc: 0.0714 - val_loss: 60.6469 - val_acc: 0.0924

Epoch 00137: val_acc did not improve
Epoch 138/1000
9828/9828 [==============================] - 2s 161us/step - loss: 50.1018 - acc: 0.0692 - val_loss: 57.9738 - val_acc: 0.0761

Epoch 00138: val_acc did not improve
Epoch 139/1000
9828/9828 [==============================] - 1s 152us/step - loss: 50.9232 - acc: 0.0706 - val_loss:


Epoch 00179: val_acc did not improve
Epoch 180/1000
9828/9828 [==============================] - 2s 200us/step - loss: 48.9418 - acc: 0.0688 - val_loss: 62.8147 - val_acc: 0.0326

Epoch 00180: val_acc did not improve
Epoch 181/1000
9828/9828 [==============================] - 2s 227us/step - loss: 49.6770 - acc: 0.0701 - val_loss: 64.6473 - val_acc: 0.0489

Epoch 00181: val_acc did not improve
Epoch 182/1000
9828/9828 [==============================] - 2s 167us/step - loss: 49.8418 - acc: 0.0680 - val_loss: 60.5132 - val_acc: 0.0924

Epoch 00182: val_acc did not improve
Epoch 183/1000
9828/9828 [==============================] - 2s 172us/step - loss: 49.7838 - acc: 0.0676 - val_loss: 56.5661 - val_acc: 0.0598

Epoch 00183: val_acc did not improve
Epoch 184/1000
9828/9828 [==============================] - 2s 227us/step - loss: 50.4343 - acc: 0.0661 - val_loss: 70.4734 - val_acc: 0.0435

Epoch 00184: val_acc did not improve
Epoch 185/1000
9828/9828 [==============================] - 2s


Epoch 00270: val_acc did not improve
Epoch 271/1000
9828/9828 [==============================] - 2s 176us/step - loss: 50.2634 - acc: 0.0704 - val_loss: 65.8646 - val_acc: 0.0326

Epoch 00271: val_acc did not improve
Epoch 272/1000
9828/9828 [==============================] - 2s 168us/step - loss: 49.5078 - acc: 0.0679 - val_loss: 55.2624 - val_acc: 0.0870

Epoch 00272: val_acc did not improve
Epoch 273/1000
9828/9828 [==============================] - 3s 260us/step - loss: 48.9664 - acc: 0.0664 - val_loss: 58.6150 - val_acc: 0.0652

Epoch 00273: val_acc did not improve
Epoch 274/1000
9828/9828 [==============================] - 2s 202us/step - loss: 49.9009 - acc: 0.0672 - val_loss: 60.0367 - val_acc: 0.0870

Epoch 00274: val_acc did not improve
Epoch 275/1000
9828/9828 [==============================] - 2s 229us/step - loss: 49.3325 - acc: 0.0698 - val_loss: 62.2634 - val_acc: 0.0543

Epoch 00275: val_acc did not improve
Epoch 276/1000
9828/9828 [==============================] - 2s

...


Epoch 00633: val_acc did not improve
Epoch 634/1000
9828/9828 [==============================] - 2s 169us/step - loss: 47.9874 - acc: 0.0699 - val_loss: 56.7106 - val_acc: 0.0652

Epoch 00634: val_acc did not improve
Epoch 635/1000
9828/9828 [==============================] - 2s 155us/step - loss: 48.0831 - acc: 0.0759 - val_loss: 49.5951 - val_acc: 0.0707

Epoch 00635: val_acc did not improve
Epoch 636/1000
9828/9828 [==============================] - 1s 147us/step - loss: 48.0702 - acc: 0.0728 - val_loss: 51.1323 - val_acc: 0.0543

Epoch 00636: val_acc did not improve
Epoch 637/1000
9828/9828 [==============================] - 1s 150us/step - loss: 49.0447 - acc: 0.0708 - val_loss: 58.5563 - val_acc: 0.0489

Epoch 00637: val_acc did not improve
Epoch 638/1000
9828/9828 [==============================] - 1s 146us/step - loss: 48.4945 - acc: 0.0654 - val_loss: 50.3759 - val_acc: 0.0761

Epoch 00638: val_acc did not improve
Epoch 639/1000
9828/9828 [==============================] - 1s


Epoch 00724: val_acc did not improve
Epoch 725/1000
9828/9828 [==============================] - 2s 159us/step - loss: 47.6703 - acc: 0.0725 - val_loss: 58.4550 - val_acc: 0.0707

Epoch 00725: val_acc did not improve
Epoch 726/1000
9828/9828 [==============================] - 1s 147us/step - loss: 48.7746 - acc: 0.0722 - val_loss: 49.8070 - val_acc: 0.0761

Epoch 00726: val_acc did not improve
Epoch 727/1000
9828/9828 [==============================] - 1s 146us/step - loss: 47.5692 - acc: 0.0724 - val_loss: 53.0531 - val_acc: 0.0707

Epoch 00727: val_acc did not improve
Epoch 728/1000
9828/9828 [==============================] - 1s 145us/step - loss: 48.0721 - acc: 0.0713 - val_loss: 52.5865 - val_acc: 0.0543

Epoch 00728: val_acc did not improve
Epoch 729/1000
9828/9828 [==============================] - 1s 149us/step - loss: 47.3735 - acc: 0.0690 - val_loss: 51.6355 - val_acc: 0.0489

Epoch 00729: val_acc did not improve
Epoch 730/1000
9828/9828 [==============================] - 1s


Epoch 00815: val_acc did not improve
Epoch 816/1000
9828/9828 [==============================] - 1s 148us/step - loss: 47.4005 - acc: 0.0700 - val_loss: 50.9984 - val_acc: 0.0326

Epoch 00816: val_acc did not improve
Epoch 817/1000
9828/9828 [==============================] - 1s 149us/step - loss: 47.8851 - acc: 0.0697 - val_loss: 52.7996 - val_acc: 0.0598

Epoch 00817: val_acc did not improve
Epoch 818/1000
9828/9828 [==============================] - 1s 148us/step - loss: 47.6174 - acc: 0.0726 - val_loss: 51.2287 - val_acc: 0.0380

Epoch 00818: val_acc did not improve
Epoch 819/1000
9828/9828 [==============================] - 1s 148us/step - loss: 47.8430 - acc: 0.0703 - val_loss: 47.4711 - val_acc: 0.0489

Epoch 00819: val_acc did not improve
Epoch 820/1000
9828/9828 [==============================] - 1s 148us/step - loss: 47.1406 - acc: 0.0743 - val_loss: 50.5829 - val_acc: 0.0489

Epoch 00820: val_acc did not improve
Epoch 821/1000
9828/9828 [==============================] - 1s


Epoch 00906: val_acc did not improve
Epoch 907/1000
9828/9828 [==============================] - 1s 145us/step - loss: 47.6605 - acc: 0.0684 - val_loss: 54.6065 - val_acc: 0.0543

Epoch 00907: val_acc did not improve
Epoch 908/1000
9828/9828 [==============================] - 1s 149us/step - loss: 48.2598 - acc: 0.0678 - val_loss: 49.7873 - val_acc: 0.0652

Epoch 00908: val_acc did not improve
Epoch 909/1000
9828/9828 [==============================] - 1s 145us/step - loss: 47.3174 - acc: 0.0679 - val_loss: 55.8228 - val_acc: 0.0543

Epoch 00909: val_acc did not improve
Epoch 910/1000
9828/9828 [==============================] - 1s 146us/step - loss: 48.1335 - acc: 0.0757 - val_loss: 52.8033 - val_acc: 0.0652

Epoch 00910: val_acc did not improve
Epoch 911/1000
9828/9828 [==============================] - 1s 146us/step - loss: 48.7581 - acc: 0.0711 - val_loss: 54.8667 - val_acc: 0.0761

Epoch 00911: val_acc did not improve
Epoch 912/1000
9828/9828 [==============================] - 1s


Epoch 00997: val_acc did not improve
Epoch 998/1000
9828/9828 [==============================] - 1s 147us/step - loss: 47.2233 - acc: 0.0698 - val_loss: 50.0864 - val_acc: 0.0326

Epoch 00998: val_acc did not improve
Epoch 999/1000
9828/9828 [==============================] - 1s 146us/step - loss: 46.9517 - acc: 0.0729 - val_loss: 51.5850 - val_acc: 0.0435

Epoch 00999: val_acc did not improve
Epoch 1000/1000
9828/9828 [==============================] - 1s 147us/step - loss: 47.2520 - acc: 0.0739 - val_loss: 51.4482 - val_acc: 0.0598

Epoch 01000: val_acc did not improve


In [23]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "C/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_C_stat = model_C.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 9828 samples, validate on 184 samples
Epoch 1/1000
9828/9828 [==============================] - 4s 370us/step - loss: 99.4100 - acc: 0.0531 - val_loss: 88.5537 - val_acc: 0.0435

Epoch 00001: val_acc improved from -inf to 0.04348, saving model to E/saved-model-001-0.0435.hdf5
Epoch 2/1000
9828/9828 [==============================] - 2s 159us/step - loss: 88.3347 - acc: 0.0428 - val_loss: 83.8689 - val_acc: 0.0272

Epoch 00002: val_acc did not improve
Epoch 3/1000
9828/9828 [==============================] - 2s 160us/step - loss: 86.0838 - acc: 0.0418 - val_loss: 80.8043 - val_acc: 0.0489

Epoch 00003: val_acc improved from 0.04348 to 0.04891, saving model to E/saved-model-003-0.0489.hdf5
Epoch 4/1000
9828/9828 [==============================] - 2s 160us/step - loss: 83.2070 - acc: 0.0416 - val_loss: 76.7561 - val_acc: 0.0489

Epoch 00004: val_acc improved from 0.04891 to 0.04891, saving model to E/saved-model-004-0.0489.hdf5
Epoch 5/1000
9828/9828 [============================

9828/9828 [==============================] - 2s 159us/step - loss: 51.8279 - acc: 0.0623 - val_loss: 58.4854 - val_acc: 0.0489

Epoch 00090: val_acc did not improve
Epoch 91/1000
9828/9828 [==============================] - 2s 159us/step - loss: 52.7474 - acc: 0.0695 - val_loss: 56.1653 - val_acc: 0.0435

Epoch 00091: val_acc did not improve
Epoch 92/1000
9828/9828 [==============================] - 2s 160us/step - loss: 52.0972 - acc: 0.0639 - val_loss: 54.7914 - val_acc: 0.0707

Epoch 00092: val_acc did not improve
Epoch 93/1000
9828/9828 [==============================] - 2s 159us/step - loss: 52.0981 - acc: 0.0645 - val_loss: 63.1119 - val_acc: 0.0217

Epoch 00093: val_acc did not improve
Epoch 94/1000
9828/9828 [==============================] - 2s 159us/step - loss: 52.2646 - acc: 0.0681 - val_loss: 57.6455 - val_acc: 0.0598

Epoch 00094: val_acc did not improve
Epoch 95/1000
9828/9828 [==============================] - 2s 162us/step - loss: 51.9029 - acc: 0.0668 - val_loss: 55.7


Epoch 00135: val_acc did not improve
Epoch 136/1000
9828/9828 [==============================] - 2s 159us/step - loss: 49.0627 - acc: 0.0747 - val_loss: 54.5381 - val_acc: 0.0707

Epoch 00136: val_acc did not improve
Epoch 137/1000
9828/9828 [==============================] - 2s 159us/step - loss: 48.5170 - acc: 0.0715 - val_loss: 84.4285 - val_acc: 0.0217

Epoch 00137: val_acc did not improve
Epoch 138/1000
9828/9828 [==============================] - 2s 162us/step - loss: 50.7971 - acc: 0.0676 - val_loss: 47.5979 - val_acc: 0.0652

Epoch 00138: val_acc did not improve
Epoch 139/1000
9828/9828 [==============================] - 2s 176us/step - loss: 48.7012 - acc: 0.0767 - val_loss: 58.3551 - val_acc: 0.0652

Epoch 00139: val_acc did not improve
Epoch 140/1000
9828/9828 [==============================] - 2s 160us/step - loss: 48.1149 - acc: 0.0720 - val_loss: 63.3814 - val_acc: 0.0435

Epoch 00140: val_acc did not improve
Epoch 141/1000
9828/9828 [==============================] - 2s

9828/9828 [==============================] - 2s 158us/step - loss: 46.8997 - acc: 0.0773 - val_loss: 95.9732 - val_acc: 0.0652

Epoch 00181: val_acc did not improve
Epoch 182/1000
9828/9828 [==============================] - 2s 158us/step - loss: 46.1444 - acc: 0.0790 - val_loss: 101.4825 - val_acc: 0.0163

Epoch 00182: val_acc did not improve
Epoch 183/1000
9828/9828 [==============================] - 2s 160us/step - loss: 46.8056 - acc: 0.0799 - val_loss: 105.4565 - val_acc: 0.0326

Epoch 00183: val_acc did not improve
Epoch 184/1000
9828/9828 [==============================] - 2s 158us/step - loss: 46.6683 - acc: 0.0815 - val_loss: 116.1875 - val_acc: 0.0435

Epoch 00184: val_acc did not improve
Epoch 185/1000
9828/9828 [==============================] - 2s 157us/step - loss: 47.0754 - acc: 0.0782 - val_loss: 120.1192 - val_acc: 0.0598

Epoch 00185: val_acc did not improve
Epoch 186/1000
9828/9828 [==============================] - 2s 159us/step - loss: 46.5824 - acc: 0.0788 - val_l

9828/9828 [==============================] - 2s 172us/step - loss: 44.6032 - acc: 0.0831 - val_loss: 55.0246 - val_acc: 0.0815

Epoch 00270: val_acc did not improve
Epoch 271/1000
9828/9828 [==============================] - 2s 181us/step - loss: 44.8353 - acc: 0.0907 - val_loss: 80.2733 - val_acc: 0.0870

Epoch 00271: val_acc did not improve
Epoch 272/1000
9828/9828 [==============================] - 2s 168us/step - loss: 44.6474 - acc: 0.0838 - val_loss: 128.4482 - val_acc: 0.0435

Epoch 00272: val_acc did not improve
Epoch 273/1000
9828/9828 [==============================] - 2s 165us/step - loss: 44.4270 - acc: 0.0864 - val_loss: 46.5900 - val_acc: 0.0924

Epoch 00273: val_acc did not improve
Epoch 274/1000
9828/9828 [==============================] - 2s 159us/step - loss: 44.9647 - acc: 0.0852 - val_loss: 69.1396 - val_acc: 0.0326

Epoch 00274: val_acc did not improve
Epoch 275/1000
9828/9828 [==============================] - 2s 161us/step - loss: 45.2155 - acc: 0.0855 - val_loss

...


Epoch 00761: val_acc did not improve
Epoch 762/1000
9828/9828 [==============================] - 2s 159us/step - loss: 108.8460 - acc: 0.0285 - val_loss: 324.7041 - val_acc: 0.0054

Epoch 00762: val_acc did not improve
Epoch 763/1000
9828/9828 [==============================] - 2s 162us/step - loss: 108.9165 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00763: val_acc did not improve
Epoch 764/1000
9828/9828 [==============================] - 2s 162us/step - loss: 108.7927 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00764: val_acc did not improve
Epoch 765/1000
9828/9828 [==============================] - 2s 172us/step - loss: 108.8199 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00765: val_acc did not improve
Epoch 766/1000
9828/9828 [==============================] - 2s 163us/step - loss: 108.8047 - acc: 0.0293 - val_loss: 15010.0467 - val_acc: 0.0000e+00

Epoch 00766: val_acc did not improve
Epoch 767/1000
9828/9828 [====================


Epoch 00850: val_acc did not improve
Epoch 851/1000
9828/9828 [==============================] - 2s 161us/step - loss: 108.7821 - acc: 0.0287 - val_loss: 168.0803 - val_acc: 0.0109

Epoch 00851: val_acc did not improve
Epoch 852/1000
9828/9828 [==============================] - 2s 157us/step - loss: 108.8166 - acc: 0.0299 - val_loss: 103.1233 - val_acc: 0.0543

Epoch 00852: val_acc did not improve
Epoch 853/1000
9828/9828 [==============================] - 2s 158us/step - loss: 108.8296 - acc: 0.0287 - val_loss: 110.0514 - val_acc: 0.0109

Epoch 00853: val_acc did not improve
Epoch 854/1000
9828/9828 [==============================] - 2s 168us/step - loss: 108.7689 - acc: 0.0287 - val_loss: 161.4385 - val_acc: 0.0380

Epoch 00854: val_acc did not improve
Epoch 855/1000
9828/9828 [==============================] - 2s 175us/step - loss: 108.8593 - acc: 0.0284 - val_loss: 97.5222 - val_acc: 0.0380

Epoch 00855: val_acc did not improve
Epoch 856/1000
9828/9828 [===========================

9828/9828 [==============================] - 2s 160us/step - loss: 108.7746 - acc: 0.0287 - val_loss: 728.8499 - val_acc: 0.0109

Epoch 00896: val_acc did not improve
Epoch 897/1000
9828/9828 [==============================] - 2s 160us/step - loss: 108.8005 - acc: 0.0287 - val_loss: 109.3652 - val_acc: 0.0380

Epoch 00897: val_acc did not improve
Epoch 898/1000
9828/9828 [==============================] - 2s 158us/step - loss: 108.8043 - acc: 0.0273 - val_loss: 247.8211 - val_acc: 0.0707

Epoch 00898: val_acc did not improve
Epoch 899/1000
9828/9828 [==============================] - 2s 163us/step - loss: 108.8137 - acc: 0.0289 - val_loss: 99.2122 - val_acc: 0.0489

Epoch 00899: val_acc did not improve
Epoch 900/1000
9828/9828 [==============================] - 2s 159us/step - loss: 108.8594 - acc: 0.0287 - val_loss: 127.8278 - val_acc: 0.0380

Epoch 00900: val_acc did not improve
Epoch 901/1000
9828/9828 [==============================] - 2s 158us/step - loss: 108.8404 - acc: 0.0292 -

9828/9828 [==============================] - 2s 161us/step - loss: 108.8304 - acc: 0.0298 - val_loss: 101.5751 - val_acc: 0.0489

Epoch 00941: val_acc did not improve
Epoch 942/1000
9828/9828 [==============================] - 2s 161us/step - loss: 108.8458 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00942: val_acc did not improve
Epoch 943/1000
9828/9828 [==============================] - 2s 164us/step - loss: 108.7882 - acc: 0.0287 - val_loss: 102.1085 - val_acc: 0.0326

Epoch 00943: val_acc did not improve
Epoch 944/1000
9828/9828 [==============================] - 2s 169us/step - loss: 108.7950 - acc: 0.0299 - val_loss: 854.0777 - val_acc: 0.0000e+00

Epoch 00944: val_acc did not improve
Epoch 945/1000
9828/9828 [==============================] - 2s 161us/step - loss: 108.8836 - acc: 0.0297 - val_loss: 1011.7632 - val_acc: 0.0000e+00

Epoch 00945: val_acc did not improve
Epoch 946/1000
9828/9828 [==============================] - 2s 226us/step - loss: 108.8631 - acc

9828/9828 [==============================] - 2s 164us/step - loss: 108.9134 - acc: 0.0266 - val_loss: 488.7873 - val_acc: 0.0054

Epoch 00986: val_acc did not improve
Epoch 987/1000
9828/9828 [==============================] - 2s 159us/step - loss: 108.8137 - acc: 0.0291 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00987: val_acc did not improve
Epoch 988/1000
9828/9828 [==============================] - 2s 161us/step - loss: 108.8031 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00988: val_acc did not improve
Epoch 989/1000
9828/9828 [==============================] - 2s 156us/step - loss: 108.8133 - acc: 0.0287 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00989: val_acc did not improve
Epoch 990/1000
9828/9828 [==============================] - 2s 159us/step - loss: 108.9015 - acc: 0.0301 - val_loss: 273.4674 - val_acc: 0.0272

Epoch 00990: val_acc did not improve
Epoch 991/1000
9828/9828 [==============================] - 2s 156us/step - loss: 108.7754 - acc: 0.0287 

In [24]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "D/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_D_stat = model_D.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 9828 samples, validate on 184 samples
Epoch 1/1000
9828/9828 [==============================] - 4s 431us/step - loss: 105.8509 - acc: 0.0412 - val_loss: 91.5695 - val_acc: 0.0489

Epoch 00001: val_acc improved from -inf to 0.04891, saving model to F/saved-model-001-0.0489.hdf5
Epoch 2/1000
9828/9828 [==============================] - 2s 180us/step - loss: 76.2863 - acc: 0.0461 - val_loss: 84.6366 - val_acc: 0.0543

Epoch 00002: val_acc improved from 0.04891 to 0.05435, saving model to F/saved-model-002-0.0543.hdf5
Epoch 3/1000
9828/9828 [==============================] - 2s 175us/step - loss: 71.7308 - acc: 0.0465 - val_loss: 71.8293 - val_acc: 0.0598

Epoch 00003: val_acc improved from 0.05435 to 0.05978, saving model to F/saved-model-003-0.0598.hdf5
Epoch 4/1000
9828/9828 [==============================] - 2s 178us/step - loss: 69.3173 - acc: 0.0512 - val_loss: 78.4789 - val_acc: 0.0326

Epoch 00004: val_acc did not improve
Epoch 5/1000
9828/9828 [===========================

9828/9828 [==============================] - 2s 178us/step - loss: 50.1516 - acc: 0.0642 - val_loss: 52.7708 - val_acc: 0.0380

Epoch 00091: val_acc did not improve
Epoch 92/1000
9828/9828 [==============================] - 2s 194us/step - loss: 50.4516 - acc: 0.0674 - val_loss: 56.2733 - val_acc: 0.0380

Epoch 00092: val_acc did not improve
Epoch 93/1000
9828/9828 [==============================] - 2s 194us/step - loss: 49.6508 - acc: 0.0708 - val_loss: 55.6855 - val_acc: 0.0489

Epoch 00093: val_acc did not improve
Epoch 94/1000
9828/9828 [==============================] - 2s 181us/step - loss: 49.9597 - acc: 0.0662 - val_loss: 57.9339 - val_acc: 0.0435

Epoch 00094: val_acc did not improve
Epoch 95/1000
9828/9828 [==============================] - 2s 172us/step - loss: 50.4907 - acc: 0.0730 - val_loss: 53.6864 - val_acc: 0.0598

Epoch 00095: val_acc did not improve
Epoch 96/1000
9828/9828 [==============================] - 2s 173us/step - loss: 50.6508 - acc: 0.0678 - val_loss: 57.2

9828/9828 [==============================] - 2s 192us/step - loss: 49.5537 - acc: 0.0699 - val_loss: 55.4121 - val_acc: 0.0380

Epoch 00136: val_acc did not improve
Epoch 137/1000
9828/9828 [==============================] - 2s 175us/step - loss: 49.7470 - acc: 0.0684 - val_loss: 51.8522 - val_acc: 0.0489

Epoch 00137: val_acc did not improve
Epoch 138/1000
9828/9828 [==============================] - 2s 183us/step - loss: 50.7863 - acc: 0.0640 - val_loss: 56.9340 - val_acc: 0.0543

Epoch 00138: val_acc did not improve
Epoch 139/1000
9828/9828 [==============================] - 2s 172us/step - loss: 49.6441 - acc: 0.0663 - val_loss: 58.1876 - val_acc: 0.0489

Epoch 00139: val_acc did not improve
Epoch 140/1000
9828/9828 [==============================] - 2s 171us/step - loss: 49.7685 - acc: 0.0643 - val_loss: 55.4397 - val_acc: 0.0598

Epoch 00140: val_acc did not improve
Epoch 141/1000
9828/9828 [==============================] - 2s 176us/step - loss: 50.4007 - acc: 0.0664 - val_loss:


Epoch 00181: val_acc did not improve
Epoch 182/1000
9828/9828 [==============================] - 2s 174us/step - loss: 48.2679 - acc: 0.0690 - val_loss: 54.7286 - val_acc: 0.0489

Epoch 00182: val_acc did not improve
Epoch 183/1000
9828/9828 [==============================] - 2s 186us/step - loss: 49.9932 - acc: 0.0714 - val_loss: 53.4070 - val_acc: 0.0489

Epoch 00183: val_acc did not improve
Epoch 184/1000
9828/9828 [==============================] - 2s 186us/step - loss: 48.8650 - acc: 0.0696 - val_loss: 48.4285 - val_acc: 0.0543

Epoch 00184: val_acc did not improve
Epoch 185/1000
9828/9828 [==============================] - 2s 173us/step - loss: 49.5307 - acc: 0.0653 - val_loss: 49.6162 - val_acc: 0.0489

Epoch 00185: val_acc did not improve
Epoch 186/1000
9828/9828 [==============================] - 2s 170us/step - loss: 51.4102 - acc: 0.0647 - val_loss: 52.5399 - val_acc: 0.0489

Epoch 00186: val_acc did not improve
Epoch 187/1000
9828/9828 [==============================] - 2s


Epoch 00272: val_acc did not improve
Epoch 273/1000
9828/9828 [==============================] - 2s 172us/step - loss: 51.5732 - acc: 0.0656 - val_loss: 50.9423 - val_acc: 0.0435

Epoch 00273: val_acc did not improve
Epoch 274/1000
9828/9828 [==============================] - 2s 174us/step - loss: 50.5435 - acc: 0.0662 - val_loss: 52.5506 - val_acc: 0.0598

Epoch 00274: val_acc did not improve
Epoch 275/1000
9828/9828 [==============================] - 2s 169us/step - loss: 50.9559 - acc: 0.0665 - val_loss: 59.7211 - val_acc: 0.0435

Epoch 00275: val_acc did not improve
Epoch 276/1000
9828/9828 [==============================] - 2s 174us/step - loss: 51.7804 - acc: 0.0650 - val_loss: 60.1169 - val_acc: 0.0543

Epoch 00276: val_acc did not improve
Epoch 277/1000
9828/9828 [==============================] - 2s 175us/step - loss: 49.6271 - acc: 0.0686 - val_loss: 50.9166 - val_acc: 0.0652

Epoch 00277: val_acc did not improve
Epoch 278/1000
9828/9828 [==============================] - 2s

...


Epoch 00727: val_acc did not improve
Epoch 728/1000
9828/9828 [==============================] - 3s 308us/step - loss: 48.1088 - acc: 0.0738 - val_loss: 45.2991 - val_acc: 0.0380

Epoch 00728: val_acc did not improve
Epoch 729/1000
9828/9828 [==============================] - 2s 246us/step - loss: 46.7038 - acc: 0.0742 - val_loss: 45.4278 - val_acc: 0.0326

Epoch 00729: val_acc did not improve
Epoch 730/1000
9828/9828 [==============================] - 2s 254us/step - loss: 49.1574 - acc: 0.0650 - val_loss: 47.1177 - val_acc: 0.0598

Epoch 00730: val_acc did not improve
Epoch 731/1000
9828/9828 [==============================] - 2s 248us/step - loss: 48.2057 - acc: 0.0685 - val_loss: 46.0167 - val_acc: 0.0870

Epoch 00731: val_acc did not improve
Epoch 732/1000
9828/9828 [==============================] - 3s 276us/step - loss: 47.0819 - acc: 0.0680 - val_loss: 49.8841 - val_acc: 0.0435

Epoch 00732: val_acc did not improve
Epoch 733/1000
9828/9828 [==============================] - 3s


Epoch 00818: val_acc did not improve
Epoch 819/1000
9828/9828 [==============================] - 2s 176us/step - loss: 47.4181 - acc: 0.0665 - val_loss: 50.8907 - val_acc: 0.0543

Epoch 00819: val_acc did not improve
Epoch 820/1000
9828/9828 [==============================] - 2s 176us/step - loss: 48.6149 - acc: 0.0693 - val_loss: 45.3368 - val_acc: 0.0652

Epoch 00820: val_acc did not improve
Epoch 821/1000
9828/9828 [==============================] - 2s 183us/step - loss: 47.0452 - acc: 0.0674 - val_loss: 53.7543 - val_acc: 0.0598

Epoch 00821: val_acc did not improve
Epoch 822/1000
9828/9828 [==============================] - 2s 181us/step - loss: 47.5391 - acc: 0.0679 - val_loss: 49.1714 - val_acc: 0.0652

Epoch 00822: val_acc did not improve
Epoch 823/1000
9828/9828 [==============================] - 2s 185us/step - loss: 47.0886 - acc: 0.0746 - val_loss: 51.9081 - val_acc: 0.0707

Epoch 00823: val_acc did not improve
Epoch 824/1000
9828/9828 [==============================] - 2s

9828/9828 [==============================] - 2s 179us/step - loss: 46.8920 - acc: 0.0733 - val_loss: 46.2535 - val_acc: 0.0652

Epoch 00909: val_acc did not improve
Epoch 910/1000
9828/9828 [==============================] - 2s 178us/step - loss: 46.0175 - acc: 0.0719 - val_loss: 47.5994 - val_acc: 0.0652

Epoch 00910: val_acc did not improve
Epoch 911/1000
9828/9828 [==============================] - 2s 175us/step - loss: 47.2002 - acc: 0.0649 - val_loss: 51.7775 - val_acc: 0.0707

Epoch 00911: val_acc did not improve
Epoch 912/1000
9828/9828 [==============================] - 2s 174us/step - loss: 48.3510 - acc: 0.0673 - val_loss: 50.9101 - val_acc: 0.0435

Epoch 00912: val_acc did not improve
Epoch 913/1000
9828/9828 [==============================] - 2s 180us/step - loss: 47.6758 - acc: 0.0704 - val_loss: 45.7280 - val_acc: 0.0707

Epoch 00913: val_acc did not improve
Epoch 914/1000
9828/9828 [==============================] - 2s 173us/step - loss: 47.0454 - acc: 0.0702 - val_loss:


Epoch 00954: val_acc did not improve
Epoch 955/1000
9828/9828 [==============================] - 2s 175us/step - loss: 46.7647 - acc: 0.0706 - val_loss: 47.8036 - val_acc: 0.0652

Epoch 00955: val_acc did not improve
Epoch 956/1000
9828/9828 [==============================] - 2s 172us/step - loss: 48.0857 - acc: 0.0708 - val_loss: 59.8154 - val_acc: 0.0652

Epoch 00956: val_acc did not improve
Epoch 957/1000
9828/9828 [==============================] - 2s 171us/step - loss: 47.5557 - acc: 0.0659 - val_loss: 46.1077 - val_acc: 0.0652

Epoch 00957: val_acc did not improve
Epoch 958/1000
9828/9828 [==============================] - 2s 173us/step - loss: 50.7479 - acc: 0.0648 - val_loss: 60.9711 - val_acc: 0.0435

Epoch 00958: val_acc did not improve
Epoch 959/1000
9828/9828 [==============================] - 2s 211us/step - loss: 48.8576 - acc: 0.0666 - val_loss: 47.8860 - val_acc: 0.0652

Epoch 00959: val_acc did not improve
Epoch 960/1000
9828/9828 [==============================] - 2s

9828/9828 [==============================] - 2s 179us/step - loss: 46.8076 - acc: 0.0745 - val_loss: 48.1779 - val_acc: 0.0652

Epoch 01000: val_acc did not improve


In [25]:
from sklearn.utils import class_weight
class_weight = class_weight.compute_class_weight('balanced'
                                               ,np.unique(y_train_2017)
                                               ,y_train_2017)

filepath = "E/saved-model-{epoch:03d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
model_E_stat = model_E.fit(x_train_2017, y_train_2017, 
                              batch_size=128, 
                              epochs=1000, 
                              validation_data=(x_test_2018, y_test_2018),
                              callbacks=[checkpoint],
                              shuffle=True,
                              class_weight=class_weight
                             )

Train on 9828 samples, validate on 184 samples
Epoch 1/1000
9828/9828 [==============================] - 4s 436us/step - loss: 123.5529 - acc: 0.0440 - val_loss: 98.6500 - val_acc: 0.0435

Epoch 00001: val_acc improved from -inf to 0.04348, saving model to G/saved-model-001-0.0435.hdf5
Epoch 2/1000
9828/9828 [==============================] - 2s 205us/step - loss: 93.6998 - acc: 0.0373 - val_loss: 90.4240 - val_acc: 0.0435

Epoch 00002: val_acc improved from 0.04348 to 0.04348, saving model to G/saved-model-002-0.0435.hdf5
Epoch 3/1000
9828/9828 [==============================] - 2s 179us/step - loss: 86.3699 - acc: 0.0402 - val_loss: 84.5774 - val_acc: 0.0435

Epoch 00003: val_acc did not improve
Epoch 4/1000
9828/9828 [==============================] - 2s 224us/step - loss: 82.0804 - acc: 0.0443 - val_loss: 79.5902 - val_acc: 0.0652

Epoch 00004: val_acc improved from 0.04348 to 0.06522, saving model to G/saved-model-004-0.0652.hdf5
Epoch 5/1000
9828/9828 [===========================

9828/9828 [==============================] - 2s 242us/step - loss: 65.3550 - acc: 0.0528 - val_loss: 66.9607 - val_acc: 0.0761

Epoch 00044: val_acc did not improve
Epoch 45/1000
9828/9828 [==============================] - 2s 189us/step - loss: 66.1465 - acc: 0.0506 - val_loss: 65.7559 - val_acc: 0.0435

Epoch 00045: val_acc did not improve
Epoch 46/1000
9828/9828 [==============================] - 2s 214us/step - loss: 65.2814 - acc: 0.0554 - val_loss: 68.0072 - val_acc: 0.0489

Epoch 00046: val_acc did not improve
Epoch 47/1000
9828/9828 [==============================] - 2s 252us/step - loss: 64.3307 - acc: 0.0476 - val_loss: 65.8036 - val_acc: 0.0435

Epoch 00047: val_acc did not improve
Epoch 48/1000
9828/9828 [==============================] - 2s 240us/step - loss: 65.1169 - acc: 0.0526 - val_loss: 66.9737 - val_acc: 0.0543

Epoch 00048: val_acc did not improve
Epoch 49/1000
9828/9828 [==============================] - 2s 187us/step - loss: 64.6538 - acc: 0.0518 - val_loss: 68.5

9828/9828 [==============================] - 1s 134us/step - loss: 59.1928 - acc: 0.0528 - val_loss: 61.4400 - val_acc: 0.0707

Epoch 00090: val_acc did not improve
Epoch 91/1000
9828/9828 [==============================] - 1s 135us/step - loss: 59.4215 - acc: 0.0532 - val_loss: 62.9582 - val_acc: 0.0707

Epoch 00091: val_acc did not improve
Epoch 92/1000
9828/9828 [==============================] - 1s 131us/step - loss: 59.6928 - acc: 0.0561 - val_loss: 61.0474 - val_acc: 0.0652

Epoch 00092: val_acc did not improve
Epoch 93/1000
9828/9828 [==============================] - 1s 135us/step - loss: 59.2689 - acc: 0.0602 - val_loss: 61.4491 - val_acc: 0.0652

Epoch 00093: val_acc did not improve
Epoch 94/1000
9828/9828 [==============================] - 2s 164us/step - loss: 60.3024 - acc: 0.0543 - val_loss: 63.7463 - val_acc: 0.0707

Epoch 00094: val_acc did not improve
Epoch 95/1000
9828/9828 [==============================] - 1s 132us/step - loss: 60.0911 - acc: 0.0546 - val_loss: 60.6


Epoch 00134: val_acc did not improve
Epoch 135/1000
9828/9828 [==============================] - 2s 224us/step - loss: 57.7409 - acc: 0.0570 - val_loss: 66.8820 - val_acc: 0.0435

Epoch 00135: val_acc did not improve
Epoch 136/1000
9828/9828 [==============================] - 3s 287us/step - loss: 55.7645 - acc: 0.0602 - val_loss: 58.3394 - val_acc: 0.0761

Epoch 00136: val_acc did not improve
Epoch 137/1000
9828/9828 [==============================] - 3s 263us/step - loss: 55.1355 - acc: 0.0609 - val_loss: 58.9903 - val_acc: 0.0489

Epoch 00137: val_acc did not improve
Epoch 138/1000
9828/9828 [==============================] - 2s 248us/step - loss: 55.8529 - acc: 0.0588 - val_loss: 60.8004 - val_acc: 0.0543

Epoch 00138: val_acc did not improve
Epoch 139/1000
9828/9828 [==============================] - 1s 147us/step - loss: 56.1049 - acc: 0.0548 - val_loss: 60.1413 - val_acc: 0.0870

Epoch 00139: val_acc did not improve
Epoch 140/1000
9828/9828 [==============================] - 2s

9828/9828 [==============================] - 2s 235us/step - loss: 54.4329 - acc: 0.0637 - val_loss: 52.9267 - val_acc: 0.0815

Epoch 00225: val_acc did not improve
Epoch 226/1000
9828/9828 [==============================] - 2s 203us/step - loss: 54.5010 - acc: 0.0638 - val_loss: 60.3759 - val_acc: 0.0489

Epoch 00226: val_acc did not improve
Epoch 227/1000
9828/9828 [==============================] - 3s 263us/step - loss: 52.5033 - acc: 0.0674 - val_loss: 53.0202 - val_acc: 0.0707

Epoch 00227: val_acc did not improve
Epoch 228/1000
9828/9828 [==============================] - 2s 203us/step - loss: 52.6687 - acc: 0.0626 - val_loss: 59.7433 - val_acc: 0.0380

Epoch 00228: val_acc did not improve
Epoch 229/1000
9828/9828 [==============================] - 2s 220us/step - loss: 54.0003 - acc: 0.0648 - val_loss: 53.2295 - val_acc: 0.0489

Epoch 00229: val_acc did not improve
Epoch 230/1000
9828/9828 [==============================] - 2s 239us/step - loss: 52.9099 - acc: 0.0660 - val_loss:

...

9828/9828 [==============================] - 1s 140us/step - loss: 51.1093 - acc: 0.0675 - val_loss: 48.0843 - val_acc: 0.0543

Epoch 00633: val_acc did not improve
Epoch 634/1000
9828/9828 [==============================] - 2s 164us/step - loss: 50.4407 - acc: 0.0670 - val_loss: 48.3062 - val_acc: 0.0761

Epoch 00634: val_acc did not improve
Epoch 635/1000
9828/9828 [==============================] - 1s 135us/step - loss: 51.2312 - acc: 0.0659 - val_loss: 48.9470 - val_acc: 0.0652

Epoch 00635: val_acc did not improve
Epoch 636/1000
9828/9828 [==============================] - 1s 137us/step - loss: 51.5243 - acc: 0.0668 - val_loss: 51.1662 - val_acc: 0.0543

Epoch 00636: val_acc did not improve
Epoch 637/1000
9828/9828 [==============================] - 1s 141us/step - loss: 51.0870 - acc: 0.0657 - val_loss: 54.6473 - val_acc: 0.0598

Epoch 00637: val_acc did not improve
Epoch 638/1000
9828/9828 [==============================] - 1s 128us/step - loss: 52.5521 - acc: 0.0631 - val_loss:


Epoch 00723: val_acc did not improve
Epoch 724/1000
9828/9828 [==============================] - 1s 137us/step - loss: 51.1562 - acc: 0.0659 - val_loss: 52.4879 - val_acc: 0.0652

Epoch 00724: val_acc did not improve
Epoch 725/1000
9828/9828 [==============================] - 1s 139us/step - loss: 50.4474 - acc: 0.0650 - val_loss: 48.5539 - val_acc: 0.0761

Epoch 00725: val_acc did not improve
Epoch 726/1000
9828/9828 [==============================] - 1s 133us/step - loss: 50.1917 - acc: 0.0653 - val_loss: 48.9067 - val_acc: 0.0543

Epoch 00726: val_acc did not improve
Epoch 727/1000
9828/9828 [==============================] - 1s 134us/step - loss: 50.4467 - acc: 0.0699 - val_loss: 51.8600 - val_acc: 0.0707

Epoch 00727: val_acc did not improve
Epoch 728/1000
9828/9828 [==============================] - 1s 131us/step - loss: 51.1799 - acc: 0.0698 - val_loss: 48.2465 - val_acc: 0.0543

Epoch 00728: val_acc did not improve
Epoch 729/1000
9828/9828 [==============================] - 1s


Epoch 00814: val_acc did not improve
Epoch 815/1000
9828/9828 [==============================] - 1s 132us/step - loss: 50.5346 - acc: 0.0663 - val_loss: 53.1032 - val_acc: 0.0707

Epoch 00815: val_acc did not improve
Epoch 816/1000
9828/9828 [==============================] - 1s 130us/step - loss: 50.4717 - acc: 0.0670 - val_loss: 52.4645 - val_acc: 0.0435

Epoch 00816: val_acc did not improve
Epoch 817/1000
9828/9828 [==============================] - 1s 129us/step - loss: 50.6196 - acc: 0.0655 - val_loss: 50.0804 - val_acc: 0.0815

Epoch 00817: val_acc did not improve
Epoch 818/1000
9828/9828 [==============================] - 1s 133us/step - loss: 51.1257 - acc: 0.0674 - val_loss: 50.3539 - val_acc: 0.0435

Epoch 00818: val_acc did not improve
Epoch 819/1000
9828/9828 [==============================] - 1s 136us/step - loss: 51.1275 - acc: 0.0657 - val_loss: 51.3654 - val_acc: 0.0326

Epoch 00819: val_acc did not improve
Epoch 820/1000
9828/9828 [==============================] - 1s


Epoch 00905: val_acc did not improve
Epoch 906/1000
9828/9828 [==============================] - 2s 156us/step - loss: 51.3687 - acc: 0.0660 - val_loss: 49.0524 - val_acc: 0.0543

Epoch 00906: val_acc did not improve
Epoch 907/1000
9828/9828 [==============================] - 2s 166us/step - loss: 50.1297 - acc: 0.0654 - val_loss: 48.5364 - val_acc: 0.1033

Epoch 00907: val_acc did not improve
Epoch 908/1000
9828/9828 [==============================] - 2s 158us/step - loss: 52.0294 - acc: 0.0639 - val_loss: 50.6481 - val_acc: 0.0489

Epoch 00908: val_acc did not improve
Epoch 909/1000
9828/9828 [==============================] - 2s 159us/step - loss: 49.7945 - acc: 0.0635 - val_loss: 52.9447 - val_acc: 0.0598

Epoch 00909: val_acc did not improve
Epoch 910/1000
9828/9828 [==============================] - 2s 177us/step - loss: 51.6486 - acc: 0.0681 - val_loss: 50.3732 - val_acc: 0.0761

Epoch 00910: val_acc did not improve
Epoch 911/1000
9828/9828 [==============================] - 2s


Epoch 00996: val_acc did not improve
Epoch 997/1000
9828/9828 [==============================] - 2s 244us/step - loss: 49.8978 - acc: 0.0728 - val_loss: 48.5387 - val_acc: 0.0707

Epoch 00997: val_acc did not improve
Epoch 998/1000
9828/9828 [==============================] - 2s 243us/step - loss: 51.6402 - acc: 0.0707 - val_loss: 54.5352 - val_acc: 0.0326

Epoch 00998: val_acc did not improve
Epoch 999/1000
9828/9828 [==============================] - 2s 204us/step - loss: 51.8676 - acc: 0.0670 - val_loss: 51.9825 - val_acc: 0.0543

Epoch 00999: val_acc did not improve
Epoch 1000/1000
9828/9828 [==============================] - 2s 204us/step - loss: 50.8913 - acc: 0.0682 - val_loss: 49.7741 - val_acc: 0.0598

Epoch 01000: val_acc did not improve


### Analysis

In [7]:
AAA = load_model('2018/saved-model-070-0.1196-A.hdf5')
BBB = load_model('2018/saved-model-107-0.1359-B.hdf5')
CCC = load_model('2018/saved-model-262-0.1087-C.hdf5')
DDD = load_model('2018/saved-model-988-0.1033-D.hdf5')
EEE = load_model('2018/saved-model-524-0.1250-E.hdf5')

In [254]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = AAA.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2018)))
print("正負1：%.4f" % (a/len(y_test_2018)))
print("正負3：%.4f" % (b/len(y_test_2018)))
print("正負5：%.4f" % (c/len(y_test_2018)))
print("正負10：%.4f" % (d/len(y_test_2018)))

答對：0.1196
正負1：0.2174
正負3：0.4511
正負5：0.5815
正負10：0.8533


In [255]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.518795070440873
mean_squared_error: 55.11087346369651
rmse: 7.423669811063562


In [256]:
Counter(sorted(p))

Counter({0.0: 1,
         1.0: 1,
         2.0: 2,
         3.0: 1,
         4.0: 3,
         8.0: 1,
         10.0: 3,
         11.0: 1,
         13.0: 2,
         14.0: 1,
         15.0: 1,
         16.0: 2,
         17.0: 1,
         18.0: 1,
         21.0: 1})

In [257]:
Counter(sorted(n))

Counter({0.0: 2,
         1.0: 3,
         2.0: 4,
         3.0: 1,
         4.0: 3,
         5.0: 2,
         7.0: 1,
         8.0: 4,
         9.0: 2,
         10.0: 5,
         11.0: 1,
         12.0: 1,
         13.0: 3,
         14.0: 2,
         15.0: 1,
         16.0: 2,
         17.0: 1,
         18.0: 1,
         21.0: 1})

In [258]:
Counter(sorted(m))

Counter({0.0: 4,
         1.0: 6,
         2.0: 4,
         3.0: 3,
         4.0: 6,
         5.0: 4,
         6.0: 5,
         7.0: 3,
         8.0: 5,
         9.0: 4,
         10.0: 5,
         11.0: 6,
         12.0: 3,
         13.0: 6,
         14.0: 5,
         15.0: 5,
         16.0: 3,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         23.0: 1,
         25.0: 1})

In [259]:
Counter(sorted(q))

Counter({0.0: 1,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         20.0: 2,
         21.0: 2,
         22.0: 1,
         23.0: 4,
         24.0: 1,
         27.0: 3,
         28.0: 1,
         36.0: 2,
         37.0: 2,
         38.0: 1,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [260]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [4, 10, 15, 22, 45, 47, 51, 57, 70, 73, 74, 77, 94, 95, 103, 108, 129, 136, 152, 168, 176, 177]
1: [4, 5, 8, 10, 13, 15, 22, 28, 34, 45, 47, 48, 49, 51, 54, 57, 70, 73, 74, 77, 81, 85, 94, 95, 103, 108, 113, 114, 115, 129, 136, 151, 152, 154, 155, 160, 168, 176, 177, 182]
3: [2, 4, 5, 8, 10, 11, 13, 15, 16, 22, 28, 34, 39, 42, 45, 46, 47, 48, 49, 51, 53, 54, 57, 58, 59, 62, 65, 68, 69, 70, 71, 73, 74, 77, 78, 81, 83, 85, 86, 92, 93, 94, 95, 96, 98, 99, 101, 103, 105, 108, 113, 114, 115, 116, 117, 120, 121, 124, 125, 129, 132, 133, 136, 142, 144, 151, 152, 153, 154, 155, 156, 160, 161, 162, 166, 168, 169, 172, 176, 177, 180, 181, 182]
NA: [0, 3, 7, 9, 18, 19, 21, 25, 32, 35, 38, 41, 44, 52, 76, 82, 87, 104, 106, 110, 111, 122, 126, 135, 149, 167, 179]


In [261]:
AA = testing_2018.copy()
AA['Prediction'] = np.round(predictions)
AA['Difference'] = AA.SixthYear_HR - AA.Prediction

In [ ]:
BB = testing_2018.copy()
BB['Prediction'] = np.round(predictions)
BB['Difference'] = BB.SixthYear_HR - BB.Prediction

In [269]:
CC = testing_2018.copy()
CC['Prediction'] = np.round(predictions)
CC['Difference'] = CC.SixthYear_HR - CC.Prediction

In [270]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = BBB.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2018)))
print("正負1：%.4f" % (a/len(y_test_2018)))
print("正負3：%.4f" % (b/len(y_test_2018)))
print("正負5：%.4f" % (c/len(y_test_2018)))
print("正負10：%.4f" % (d/len(y_test_2018)))

答對：0.1359
正負1：0.2391
正負3：0.4130
正負5：0.5652
正負10：0.8152


In [271]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.783072621926017
mean_squared_error: 59.66272866704309
rmse: 7.724165240790948


In [272]:
Counter(sorted(p))

Counter({2.0: 2,
         3.0: 2,
         4.0: 3,
         6.0: 2,
         7.0: 1,
         9.0: 2,
         10.0: 3,
         12.0: 2,
         13.0: 1,
         14.0: 3,
         16.0: 2,
         17.0: 1,
         21.0: 1})

In [273]:
Counter(sorted(n))

Counter({1.0: 2,
         2.0: 3,
         3.0: 3,
         4.0: 3,
         5.0: 1,
         6.0: 2,
         7.0: 1,
         8.0: 2,
         9.0: 3,
         10.0: 3,
         12.0: 2,
         13.0: 5,
         14.0: 5,
         15.0: 3,
         16.0: 2,
         17.0: 1,
         18.0: 1,
         21.0: 1,
         25.0: 1})

In [274]:
Counter(sorted(m))

Counter({0.0: 4,
         1.0: 5,
         2.0: 3,
         3.0: 3,
         4.0: 5,
         5.0: 4,
         6.0: 4,
         7.0: 2,
         8.0: 5,
         9.0: 4,
         10.0: 5,
         11.0: 6,
         12.0: 3,
         13.0: 6,
         14.0: 5,
         15.0: 4,
         16.0: 3,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 1,
         25.0: 1})

In [275]:
Counter(sorted(q))

Counter({1.0: 1,
         2.0: 1,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         15.0: 2,
         20.0: 2,
         22.0: 1,
         23.0: 5,
         24.0: 1,
         25.0: 1,
         27.0: 3,
         28.0: 1,
         30.0: 1,
         33.0: 1,
         34.0: 1,
         36.0: 2,
         37.0: 2,
         38.0: 2,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [276]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [2, 10, 13, 28, 34, 45, 47, 62, 70, 94, 98, 103, 108, 113, 114, 117, 120, 121, 129, 151, 152, 168, 169, 173, 180]
1: [2, 4, 10, 11, 13, 15, 16, 28, 34, 42, 45, 46, 47, 48, 54, 57, 62, 70, 74, 85, 93, 94, 95, 98, 99, 103, 108, 113, 114, 115, 117, 120, 121, 124, 129, 151, 152, 153, 168, 169, 173, 177, 180, 182]
3: [1, 2, 4, 5, 8, 10, 11, 13, 15, 16, 22, 28, 34, 37, 39, 42, 45, 46, 47, 48, 49, 51, 53, 54, 57, 62, 65, 70, 73, 74, 77, 78, 79, 81, 83, 85, 86, 92, 93, 94, 95, 98, 99, 101, 103, 108, 113, 114, 115, 116, 117, 120, 121, 124, 125, 127, 128, 129, 142, 144, 151, 152, 153, 154, 155, 160, 162, 168, 169, 173, 176, 177, 178, 180, 181, 182]
NA: [3, 7, 9, 18, 19, 21, 25, 35, 36, 38, 41, 44, 52, 64, 66, 76, 80, 84, 87, 88, 104, 106, 110, 111, 122, 126, 135, 138, 149, 163, 165, 167, 170, 179]


In [277]:
DD = testing_2018.copy()
DD['Prediction'] = np.round(predictions)
DD['Difference'] = DD.SixthYear_HR - DD.Prediction

In [278]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = CCC.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2018)))
print("正負1：%.4f" % (a/len(y_test_2018)))
print("正負3：%.4f" % (b/len(y_test_2018)))
print("正負5：%.4f" % (c/len(y_test_2018)))
print("正負10：%.4f" % (d/len(y_test_2018)))

答對：0.1087
正負1：0.2174
正負3：0.4076
正負5：0.5870
正負10：0.8913


In [279]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.312735424741454
mean_squared_error: 47.6781914352469
rmse: 6.9049396402319765


In [280]:
Counter(sorted(p))

Counter({0.0: 2,
         1.0: 2,
         2.0: 1,
         4.0: 1,
         5.0: 2,
         6.0: 2,
         7.0: 1,
         8.0: 2,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 1,
         14.0: 2,
         18.0: 1})

In [281]:
Counter(sorted(n))

Counter({0.0: 2,
         1.0: 5,
         2.0: 2,
         3.0: 2,
         4.0: 4,
         5.0: 2,
         6.0: 2,
         7.0: 1,
         8.0: 5,
         9.0: 1,
         10.0: 1,
         11.0: 1,
         12.0: 1,
         13.0: 2,
         14.0: 2,
         15.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 1,
         21.0: 1,
         23.0: 1,
         25.0: 1})

In [282]:
Counter(sorted(m))

Counter({0.0: 4,
         1.0: 6,
         2.0: 4,
         3.0: 3,
         4.0: 6,
         5.0: 4,
         6.0: 6,
         7.0: 1,
         8.0: 5,
         9.0: 4,
         10.0: 4,
         11.0: 4,
         12.0: 2,
         13.0: 4,
         14.0: 4,
         15.0: 2,
         16.0: 3,
         17.0: 1,
         18.0: 1,
         19.0: 1,
         20.0: 1,
         21.0: 1,
         22.0: 1,
         23.0: 2,
         25.0: 1})

In [283]:
Counter(sorted(q))

Counter({3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         12.0: 1,
         20.0: 2,
         21.0: 1,
         22.0: 1,
         23.0: 4,
         27.0: 2,
         36.0: 2,
         37.0: 1,
         43.0: 1,
         48.0: 1})

In [284]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [8, 10, 15, 28, 48, 77, 78, 86, 90, 98, 115, 116, 132, 136, 155, 160, 169, 173, 176, 177]
1: [2, 5, 8, 10, 15, 16, 28, 39, 48, 49, 51, 54, 58, 77, 78, 81, 85, 86, 90, 94, 98, 115, 116, 119, 124, 132, 136, 144, 150, 151, 152, 153, 155, 156, 158, 160, 169, 173, 176, 177]
3: [2, 5, 8, 10, 11, 13, 15, 16, 20, 22, 28, 34, 39, 42, 47, 48, 49, 51, 53, 54, 58, 62, 65, 69, 70, 71, 73, 74, 77, 78, 79, 81, 83, 85, 86, 90, 93, 94, 98, 99, 101, 103, 113, 114, 115, 116, 117, 119, 120, 124, 129, 131, 132, 136, 144, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 162, 168, 169, 171, 172, 173, 176, 177, 180, 182]
NA: [9, 14, 18, 19, 21, 25, 35, 38, 41, 44, 52, 76, 87, 104, 110, 111, 122, 140, 175, 179]


In [285]:
EE = testing_2018.copy()
EE['Prediction'] = np.round(predictions)
EE['Difference'] = EE.SixthYear_HR - EE.Prediction

In [286]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = DDD.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2018)))
print("正負1：%.4f" % (a/len(y_test_2018)))
print("正負3：%.4f" % (b/len(y_test_2018)))
print("正負5：%.4f" % (c/len(y_test_2018)))
print("正負10：%.4f" % (d/len(y_test_2018)))

答對：0.1033
正負1：0.2283
正負3：0.4565
正負5：0.5870
正負10：0.9022


In [287]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.202052433853564
mean_squared_error: 47.53984231125445
rmse: 6.894914235235595


In [288]:
Counter(sorted(p))

Counter({2.0: 3,
         4.0: 2,
         5.0: 1,
         6.0: 1,
         7.0: 1,
         8.0: 3,
         10.0: 3,
         11.0: 1,
         13.0: 1,
         14.0: 1,
         15.0: 1,
         21.0: 1})

In [289]:
Counter(sorted(n))

Counter({1.0: 5,
         2.0: 3,
         3.0: 3,
         4.0: 2,
         5.0: 2,
         6.0: 3,
         7.0: 1,
         8.0: 4,
         9.0: 1,
         10.0: 4,
         11.0: 2,
         12.0: 2,
         13.0: 2,
         14.0: 2,
         15.0: 2,
         17.0: 1,
         18.0: 1,
         20.0: 1,
         21.0: 1})

In [290]:
Counter(sorted(m))

Counter({0.0: 4,
         1.0: 8,
         2.0: 5,
         3.0: 3,
         4.0: 7,
         5.0: 4,
         6.0: 6,
         7.0: 1,
         8.0: 5,
         9.0: 3,
         10.0: 5,
         11.0: 5,
         12.0: 3,
         13.0: 3,
         14.0: 6,
         15.0: 5,
         16.0: 1,
         17.0: 1,
         18.0: 2,
         19.0: 1,
         20.0: 1,
         21.0: 2,
         22.0: 1,
         23.0: 1,
         32.0: 1})

In [291]:
Counter(sorted(q))

Counter({3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 1,
         11.0: 1,
         13.0: 1,
         20.0: 2,
         23.0: 2,
         24.0: 1,
         27.0: 2,
         36.0: 2,
         38.0: 1,
         43.0: 1,
         48.0: 1})

In [292]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [5, 8, 10, 13, 16, 22, 47, 48, 54, 81, 85, 98, 113, 116, 125, 150, 158, 160, 176]
1: [2, 5, 8, 10, 13, 15, 16, 20, 22, 28, 34, 39, 47, 48, 49, 53, 54, 74, 78, 81, 85, 86, 90, 93, 94, 98, 108, 113, 115, 116, 119, 124, 125, 132, 150, 156, 158, 160, 166, 168, 173, 176]
3: [2, 5, 8, 10, 11, 13, 15, 16, 20, 22, 28, 34, 39, 46, 47, 48, 49, 50, 51, 53, 54, 55, 57, 62, 65, 69, 73, 74, 77, 78, 79, 81, 83, 85, 86, 90, 92, 93, 94, 95, 98, 99, 101, 103, 108, 113, 114, 115, 116, 117, 119, 120, 123, 124, 125, 129, 131, 132, 134, 136, 137, 142, 144, 150, 151, 152, 154, 155, 156, 158, 159, 160, 161, 162, 166, 168, 169, 171, 173, 174, 176, 177, 180, 182]
NA: [9, 18, 19, 21, 25, 38, 40, 44, 75, 76, 87, 104, 110, 111, 122, 128, 165, 179]


In [294]:
t = 0
a = 0
b = 0
c = 0
d = 0
g = []
l = []
m = []
m1 = []
n = []
n1 = []
p = []
p1 = []
q = []
q1 = []
predictions = EEE.predict(x_test_2018)
for i in range(len(y_test_2018)):
    k = np.round(predictions)[i] - y_test_2018[i]
    if abs(k) < 11:
        d += 1
        g.append(y_test_2018[i])
        if abs(k) < 6:
            c += 1
            l.append(y_test_2018[i])
            if abs(k) < 4:
                b += 1
                m.append(y_test_2018[i])
                m1.append(i)
                if abs(k) < 2:
                    a += 1
                    n.append(y_test_2018[i])
                    n1.append(i)
                    if abs(k) == 0:
                        t += 1
                        p.append(y_test_2018[i])
                        p1.append(i)
    else:
        q.append(y_test_2018[i])
        q1.append(i)
print("答對：%.4f" % (t/len(y_test_2018)))
print("正負1：%.4f" % (a/len(y_test_2018)))
print("正負3：%.4f" % (b/len(y_test_2018)))
print("正負5：%.4f" % (c/len(y_test_2018)))
print("正負10：%.4f" % (d/len(y_test_2018)))

答對：0.1250
正負1：0.2446
正負3：0.4674
正負5：0.6087
正負10：0.8750


In [295]:
print("mean_absolute_error:",mean_absolute_error(predictions,y_test_2018))
print("mean_squared_error:",mean_squared_error(predictions,y_test_2018))
print("rmse:",sqrt(mean_squared_error(predictions,y_test_2018)))

mean_absolute_error: 5.176031209852384
mean_squared_error: 47.71780347634609
rmse: 6.907807429014368


In [296]:
Counter(sorted(p))

Counter({3.0: 1,
         5.0: 2,
         7.0: 1,
         8.0: 3,
         9.0: 1,
         10.0: 3,
         11.0: 2,
         12.0: 2,
         14.0: 2,
         15.0: 1,
         18.0: 1,
         19.0: 1,
         21.0: 2,
         25.0: 1})

In [297]:
Counter(sorted(n))

Counter({2.0: 2,
         3.0: 1,
         4.0: 5,
         5.0: 2,
         6.0: 3,
         7.0: 1,
         8.0: 4,
         9.0: 3,
         10.0: 3,
         11.0: 3,
         12.0: 3,
         13.0: 1,
         14.0: 5,
         15.0: 2,
         18.0: 1,
         19.0: 1,
         21.0: 2,
         22.0: 1,
         23.0: 1,
         25.0: 1})

In [298]:
Counter(sorted(m))

Counter({0.0: 3,
         1.0: 5,
         2.0: 3,
         3.0: 3,
         4.0: 6,
         5.0: 4,
         6.0: 5,
         7.0: 2,
         8.0: 4,
         9.0: 7,
         10.0: 5,
         11.0: 8,
         12.0: 3,
         13.0: 5,
         14.0: 5,
         15.0: 5,
         16.0: 2,
         18.0: 1,
         19.0: 1,
         20.0: 3,
         21.0: 2,
         22.0: 1,
         23.0: 2,
         25.0: 1})

In [299]:
Counter(sorted(q))

Counter({1.0: 2,
         3.0: 1,
         4.0: 1,
         8.0: 1,
         9.0: 2,
         11.0: 1,
         12.0: 1,
         21.0: 1,
         23.0: 2,
         27.0: 2,
         36.0: 2,
         37.0: 2,
         38.0: 2,
         39.0: 1,
         43.0: 1,
         48.0: 1})

In [300]:
print('0:',p1)
print('1:',n1)
print('3:',m1)
print('NA:',q1)

0: [8, 10, 11, 22, 28, 54, 62, 81, 83, 86, 90, 94, 97, 108, 129, 144, 153, 154, 158, 162, 173, 174, 176]
1: [8, 10, 11, 13, 16, 22, 28, 34, 47, 51, 54, 62, 73, 81, 83, 86, 90, 92, 94, 97, 98, 99, 108, 116, 129, 131, 132, 144, 150, 152, 153, 154, 158, 159, 161, 162, 166, 169, 173, 174, 176, 177, 178, 180, 182]
3: [1, 2, 8, 10, 11, 13, 15, 16, 17, 20, 22, 28, 34, 39, 47, 48, 49, 51, 53, 54, 58, 60, 62, 65, 68, 70, 71, 72, 73, 74, 77, 78, 79, 81, 83, 85, 86, 89, 90, 92, 93, 94, 95, 97, 98, 99, 107, 108, 112, 113, 114, 115, 116, 117, 119, 124, 129, 131, 132, 133, 144, 146, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 161, 162, 164, 166, 169, 172, 173, 174, 176, 177, 178, 180, 182, 183]
NA: [0, 7, 9, 18, 19, 21, 25, 30, 35, 38, 44, 76, 87, 104, 106, 111, 128, 139, 147, 165, 167, 175, 179]


In [242]:
BB.head()

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,FifthYear_SO,FifthYear_IBB,FifthYear_HBP,FifthYear_SH,FifthYear_SF,FifthYear_GIDP,FifthYear_weight,FifthYear_height,FifthYear_age,Prediction
0,2013,108.0,296.0,46.0,84.0,14.0,0.0,17.0,51.0,0.0,...,88.0,5.0,1.0,0.0,4.0,5.0,245.0,75.0,29.0,10.0
1,2013,9.0,18.0,3.0,4.0,1.0,0.0,1.0,3.0,0.0,...,25.0,1.0,1.0,1.0,6.0,0.0,195.0,73.0,28.0,1.0
2,2013,25.0,72.0,10.0,19.0,4.0,0.0,2.0,9.0,1.0,...,46.0,0.0,1.0,1.0,1.0,2.0,223.0,70.0,28.0,1.0
3,2013,97.0,334.0,34.0,94.0,11.0,0.0,6.0,45.0,6.0,...,118.0,6.0,2.0,0.0,0.0,9.0,230.0,73.0,30.0,11.0
4,2013,152.0,626.0,64.0,177.0,31.0,2.0,5.0,52.0,35.0,...,84.0,3.0,9.0,1.0,4.0,19.0,166.0,66.0,27.0,13.0


In [339]:
BB[BB.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [340]:
DD[DD.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [341]:
EE[EE.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [342]:
AA[AA.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference


In [343]:
CC[CC.Prediction > 39]

,FirstYear_yearID,FirstYear_G,FirstYear_AB,FirstYear_R,FirstYear_H,FirstYear_2B,FirstYear_3B,FirstYear_HR,FirstYear_RBI,FirstYear_SB,...,SixthYear_SF,SixthYear_GIDP,SixthYear_nameFirst,SixthYear_nameLast,SixthYear_nameGiven,SixthYear_weight,SixthYear_height,SixthYear_age,Prediction,Difference
